In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

# File Reading

In [0]:
path = "dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/flight_data.csv"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
schema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(),True),
    StructField("count",StringType(),True)
])

In [0]:
flight_df = spark.read.format('csv').option('header','false').option('skipRows',1).schema(schema).load(path)

In [0]:
flight_df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [0]:
%fs
ls /FileStore/shared_uploads/riteshojha2002@gmail.com/

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/2015_summary.csv,2015_summary.csv,7080,1715526321000
dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/HR_Gender_Diversity___Equality.csv,HR_Gender_Diversity___Equality.csv,114732,1713266514000
dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/San_Martin_Stores_2021_New.xlsx,San_Martin_Stores_2021_New.xlsx,1455778,1713266475000
dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/flight_data.csv,flight_data.csv,7080,1715526353000


# Handling Corrupt Files


### Failfest
- fail execution if mailfomed records in dataset
### Dropmal fomed
- Drop corrupted records
### Permissive
- Set null values to all corrupted fields

In [0]:
corrupt_schema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(),True),
    StructField("count",StringType(),True),
    StructField("_corrupt_records",StringType(),True)
])

In [0]:
# Handling Corrupt Files

flight_df_corrupt = spark.read.format('csv')\
.option('header','false')\
.option("badRecordsPath","dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/")
.schema(corrupt_schema).load(path)\

# Bad record file in Json format


# Write file in spark

- Append
- Overwrite

In [0]:
df.write.format('csv')\
    .option("header","True")\
    .option("mode","overwrite")\
    .option("path","dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/write/")
    .save()

In [0]:
# Partitioning
df.repartition(3).write.format('csv')\
    .option("header","True")\
    .option("mode","overwrite")\
    .option("path","dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/write/")
    .save()

# Partition and Bucket
- Partition used when we have columns of categorys like gender, country, city
- Bucket created when we not able to create partitions.

In [0]:
df.write.format('csv')\
    .option("header","True")\
    .option("mode","overwrite")\
    .option("path","dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/write/")\
    .partitionBy(Column-Name)\
    .save()

In [0]:
dbutils.fs.ls("dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/write/")

In [0]:
df.write.format('csv')\
    .option("header","True")\
    .option("mode","overwrite")\
    .option("path","dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/write/")\
    .bucketBy(3,"ID")\
    # .save() # save not work in bucket
    .saveAsTable('table_name')


# Create Dataframe in Spark

In [0]:
data = [
    (1,2),
    (2,1),
    (3,1),
    (4,2),
    (5,1),
    (6,2),
    (7,2)
]

In [0]:
schema = ['id','num']

In [0]:
df = spark.createDataFrame(data=data,schema=schema)
df.show()

# Transformations

## Flight df

In [0]:
from pyspark.sql.functions import col, expr, lit, concat

In [0]:
flight_df.show(1)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
+-----------------+-------------------+-----+
only showing top 1 row



In [0]:
flight_df.select("DEST_COUNTRY_NAME").show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [0]:
flight_df.select(col("DEST_COUNTRY_NAME")).show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [0]:
# Expression - Write SQL statements

flight_df.select(expr("count +5")).show(5)

+-----------+
|(count + 5)|
+-----------+
|       20.0|
|        6.0|
|      349.0|
|       20.0|
|       67.0|
+-----------+
only showing top 5 rows



In [0]:
flight_df.select(col("count")+5).show(5)

+-----------+
|(count + 5)|
+-----------+
|       20.0|
|        6.0|
|      349.0|
|       20.0|
|       67.0|
+-----------+
only showing top 5 rows



In [0]:
# Four ways - String way , Col Way, Pandas Way, SQl way

flight_df.select("DEST_COUNTRY_NAME",col("DEST_COUNTRY_NAME"),flight_df['DEST_COUNTRY_NAME'],flight_df.DEST_COUNTRY_NAME).show(3)

+-----------------+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|    United States|
|    United States|    United States|    United States|    United States|
|    United States|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+-----------------+
only showing top 3 rows



In [0]:
flight_df.select(expr("DEST_COUNTRY_NAME as Destination")).show(3)

+-------------+
|  Destination|
+-------------+
|United States|
|United States|
|United States|
+-------------+
only showing top 3 rows



In [0]:
flight_df.filter((col("DEST_COUNTRY_NAME") == "India") | (col("DEST_COUNTRY_NAME")=="Egypt")).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|            Egypt|      United States|   15|
|            India|      United States|   61|
+-----------------+-------------------+-----+



In [0]:
flight_df.where(col("DEST_COUNTRY_NAME") == "United States").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|    United States|              India|   62|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
# literal - Use to assign default values
flight_df.select("*",lit("Air India").alias("Company")).show(5)


+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|  Company|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|Air India|
|    United States|            Croatia|    1|Air India|
|    United States|            Ireland|  344|Air India|
|            Egypt|      United States|   15|Air India|
|    United States|              India|   62|Air India|
+-----------------+-------------------+-----+---------+
only showing top 5 rows



In [0]:
# With Column - Update or Create New Column

flight_df.withColumn()

## Hr Df

In [0]:
hr_df = spark.read.format('csv').option('header','True').option('inferschema','True').load("dbfs:/FileStore/shared_uploads/riteshojha2002@gmail.com/HR_Gender_Diversity___Equality.csv")

In [0]:
hr_df.printSchema()

root
 |-- Employee ID: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- Emp Age: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- Hire Date: date (nullable = true)
 |-- Leave Date: date (nullable = true)
 |-- Leave Reason: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Employee Satisfaction: integer (nullable = true)
 |-- Annual Salary ($): integer (nullable = true)
 |-- Bonus ($): integer (nullable = true)
 |-- Total Compensation: integer (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Manager (Y/N): string (nullable = true)
 |-- Performance: string (nullable = true)



In [0]:
hr_df.withColumn("Full Name",concat("Name","Surname")).show(5)

+-----------+----------+-------+---------+-------+------+--------------+------------+----------+----------+------------+------+----------+---------------------+-----------------+---------+------------------+--------------------+---------------+-------------+-----------+--------------+
|Employee ID|       DOB|Emp Age|  Surname|   Name|Gender|Marital Status|      Branch| Hire Date|Leave Date|Leave Reason|Status|Department|Employee Satisfaction|Annual Salary ($)|Bonus ($)|Total Compensation|           Job Title|Job Description|Manager (Y/N)|Performance|     Full Name|
+-----------+----------+-------+---------+-------+------+--------------+------------+----------+----------+------------+------+----------+---------------------+-----------------+---------+------------------+--------------------+---------------+-------------+-----------+--------------+
|      10005|1953-10-28|     69|  Jackson| Noelia|     F|        Single|Buenos Aires|2014-01-25|      null|Not provided|Active|Production|    

In [0]:
new_df = hr_df.withColumnRenamed("Employee ID","ID")
new_df.show(5)

+-----+----------+-------+---------+-------+------+--------------+------------+----------+----------+------------+------+----------+---------------------+-----------------+---------+------------------+--------------------+---------------+-------------+-----------+
|   ID|       DOB|Emp Age|  Surname|   Name|Gender|Marital Status|      Branch| Hire Date|Leave Date|Leave Reason|Status|Department|Employee Satisfaction|Annual Salary ($)|Bonus ($)|Total Compensation|           Job Title|Job Description|Manager (Y/N)|Performance|
+-----+----------+-------+---------+-------+------+--------------+------------+----------+----------+------------+------+----------+---------------------+-----------------+---------+------------------+--------------------+---------------+-------------+-----------+
|10005|1953-10-28|     69|  Jackson| Noelia|     F|        Single|Buenos Aires|2014-01-25|      null|Not provided|Active|Production|                    3|            43241|     4757|             47998|Prod

In [0]:
hr_df.withColumn("Emp Age",col("Emp Age").cast("string")).printSchema()

root
 |-- Employee ID: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- Emp Age: string (nullable = true)
 |-- Surname: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- Hire Date: date (nullable = true)
 |-- Leave Date: date (nullable = true)
 |-- Leave Reason: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Employee Satisfaction: integer (nullable = true)
 |-- Annual Salary ($): integer (nullable = true)
 |-- Bonus ($): integer (nullable = true)
 |-- Total Compensation: integer (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Manager (Y/N): string (nullable = true)
 |-- Performance: string (nullable = true)



# If Else ( When )

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

emp_schema = ['id','name','age','salary','country','dept']
emp_df = spark.createDataFrame(data = emp_data, schema = emp_schema)
emp_df.show()


+----+-------+----+------+-------+-----------+
|  id|   name| age|salary|country|       dept|
+----+-------+----+------+-------+-----------+
|   1| manish|  26| 20000|  india|         IT|
|   2|  rahul|null| 40000|germany|engineering|
|   3|  pawan|  12| 60000|  india|      sales|
|   4|roshini|  44|  null|     uk|engineering|
|   5|raushan|  35| 70000|  india|      sales|
|   6|   null|  29|200000|     uk|         IT|
|   7|   adam|  37| 65000|     us|         IT|
|   8|  chris|  16| 40000|     us|      sales|
|null|   null|null|  null|   null|       null|
|   7|   adam|  37| 65000|     us|         IT|
+----+-------+----+------+-------+-----------+



In [0]:
emp_df.withColumn("adult",when(col("age")<18,"No").when(col("age")>18,"Yes").otherwise("Novalue")).show()

+----+-------+----+------+-------+-----------+-------+
|  id|   name| age|salary|country|       dept|  adult|
+----+-------+----+------+-------+-----------+-------+
|   1| manish|  26| 20000|  india|         IT|    Yes|
|   2|  rahul|null| 40000|germany|engineering|Novalue|
|   3|  pawan|  12| 60000|  india|      sales|     No|
|   4|roshini|  44|  null|     uk|engineering|    Yes|
|   5|raushan|  35| 70000|  india|      sales|    Yes|
|   6|   null|  29|200000|     uk|         IT|    Yes|
|   7|   adam|  37| 65000|     us|         IT|    Yes|
|   8|  chris|  16| 40000|     us|      sales|     No|
|null|   null|null|  null|   null|       null|Novalue|
|   7|   adam|  37| 65000|     us|         IT|    Yes|
+----+-------+----+------+-------+-----------+-------+



In [0]:
emp_df.withColumn("age",when(col("age").isNull(),lit(19)).otherwise(col("age")))\
    .withColumn("adult",when(col("age")>18,"yes").otherwise("No")).show()

+----+-------+---+------+-------+-----------+-----+
|  id|   name|age|salary|country|       dept|adult|
+----+-------+---+------+-------+-----------+-----+
|   1| manish| 26| 20000|  india|         IT|  yes|
|   2|  rahul| 19| 40000|germany|engineering|  yes|
|   3|  pawan| 12| 60000|  india|      sales|   No|
|   4|roshini| 44|  null|     uk|engineering|  yes|
|   5|raushan| 35| 70000|  india|      sales|  yes|
|   6|   null| 29|200000|     uk|         IT|  yes|
|   7|   adam| 37| 65000|     us|         IT|  yes|
|   8|  chris| 16| 40000|     us|      sales|   No|
|null|   null| 19|  null|   null|       null|  yes|
|   7|   adam| 37| 65000|     us|         IT|  yes|
+----+-------+---+------+-------+-----------+-----+



## Unique and Sorted

In [0]:
data=[(10 ,'Anil',50000, 18),
(11 ,'Vikas',75000,  16),
(12 ,'Nisha',40000,  18),
(13 ,'Nidhi',60000,  17),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  18),
(16 ,'Rajesh',90000, 10),
(17 ,'Raman',55000, 16),
(18 ,'Sam',65000,   17),
(15 ,'Mohit',45000,  18),
(13 ,'Nidhi',60000,  17),      
(14 ,'Priya',90000,  18),  
(18 ,'Sam',65000,   17)
     ]

data_schema = ['id','name','salary','mngr_id']

data_df = spark.createDataFrame(data=data,schema = data_schema)

In [0]:
data_df.distinct().count()

Out[6]: 10

In [0]:
data_df.select('id','name').distinct().count()

Out[7]: 9

In [0]:
data_df.drop_duplicates(['id','name','salary','mngr_id']).show()

+---+------+------+-------+
| id|  name|salary|mngr_id|
+---+------+------+-------+
| 10|  Anil| 50000|     18|
| 12| Nisha| 40000|     18|
| 11| Vikas| 75000|     16|
| 13| Nidhi| 60000|     17|
| 15| Mohit| 45000|     18|
| 14| Priya| 80000|     18|
| 16|Rajesh| 90000|     10|
| 17| Raman| 55000|     16|
| 18|   Sam| 65000|     17|
| 14| Priya| 90000|     18|
+---+------+------+-------+



In [0]:
data_df.sort(col('name').desc()).show()

+---+------+------+-------+
| id|  name|salary|mngr_id|
+---+------+------+-------+
| 11| Vikas| 75000|     16|
| 18|   Sam| 65000|     17|
| 18|   Sam| 65000|     17|
| 17| Raman| 55000|     16|
| 16|Rajesh| 90000|     10|
| 14| Priya| 80000|     18|
| 14| Priya| 90000|     18|
| 12| Nisha| 40000|     18|
| 13| Nidhi| 60000|     17|
| 13| Nidhi| 60000|     17|
| 15| Mohit| 45000|     18|
| 15| Mohit| 45000|     18|
| 10|  Anil| 50000|     18|
+---+------+------+-------+



# JOINS

In [0]:
customer_data = [(1,'manish','patna',"30-05-2022"),
(2,'vikash','kolkata',"12-03-2023"),
(3,'nikita','delhi',"25-06-2023"),
(4,'rahul','ranchi',"24-03-2023"),
(5,'mahesh','jaipur',"22-03-2023"),
(6,'prantosh','kolkata',"18-10-2022"),
(7,'raman','patna',"30-12-2022"),
(8,'prakash','ranchi',"24-02-2023"),
(9,'ragini','kolkata',"03-03-2023"),
(10,'raushan','jaipur',"05-02-2023")]

customer_schema=['customer_id','customer_name','address','date_of_joining']
customer_df = spark.createDataFrame(data = customer_data, schema = customer_schema)

sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

sales_schema=['customer_id','product_id','quantity','date_of_purchase']
sales_df = spark.createDataFrame(data= sales_data, schema = sales_schema)

product_data = [(1, 'fanta',20),
(2, 'dew',22),
(5, 'sprite',40),
(7, 'redbull',100),
(12,'mazza',45),
(22,'coke',27),
(25,'limca',21),
(27,'pepsi',14),
(56,'sting',10)]

product_schema=['id','name','price']
product_df = spark.createDataFrame(data = product_data, schema = product_schema)

In [0]:
customer_df.join(sales_df,customer_df['customer_id'] == sales_df['customer_id'],'inner').show()
customer_df.join(sales_df,customer_df['customer_id'] == sales_df['customer_id'],'left').show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15

In [0]:
sales_df.join(product_df,sales_df['product_id'] == product_df['id'],'right').show()

+-----------+----------+--------+----------------+---+-------+-----+
|customer_id|product_id|quantity|date_of_purchase| id|   name|price|
+-----------+----------+--------+----------------+---+-------+-----+
|          2|         1|      12|      15-06-2023|  1|  fanta|   20|
|       null|      null|    null|            null|  2|    dew|   22|
|          9|         5|       6|      03-03-2023|  5| sprite|   40|
|          2|         5|       3|      01-06-2023|  5| sprite|   40|
|       null|      null|    null|            null|  7|redbull|  100|
|         11|        12|      76|      12-03-2023| 12|  mazza|   45|
|          5|        12|       5|      15-04-2023| 12|  mazza|   45|
|          7|        22|       4|      03-02-2023| 22|   coke|   27|
|          5|        22|       1|      22-03-2023| 22|   coke|   27|
|          1|        22|      10|      01-06-2022| 22|   coke|   27|
|       null|      null|    null|            null| 25|  limca|   21|
|          1|        27|       5| 

In [0]:
customer_df.join(sales_df,customer_df['customer_id'] == sales_df['customer_id'],'outer').show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          3|       nikita|  delhi|     25-06-2023|       null|      null|    null|            null|
|          4|        rahul| ranchi|     24-03-2023|       null|      null|    null|        

In [0]:
customer_df.join(sales_df,customer_df['customer_id'] == sales_df['customer_id'],'left_anti').show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          3|       nikita|  delhi|     25-06-2023|
|          4|        rahul| ranchi|     24-03-2023|
|          6|     prantosh|kolkata|     18-10-2022|
|          8|      prakash| ranchi|     24-02-2023|
|         10|      raushan| jaipur|     05-02-2023|
+-----------+-------------+-------+---------------+



In [0]:
customer_df.crossJoin(sales_df).show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          2|         5|       3|      01-06-2023|
|          1|       manish|  patna|     30-05-2022|          5|        22|       1|      22-03-2023|
|          1|       manish|  patna|     30-05-2022|          7|        22|       4|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          9|         5|       6|      03-03-2023|
|          1|       manish|  patna|     30-05-2022|          2|         1|      12|      15

# Window Funtions

In [0]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

emp_schema = ['id','name','sal','dept','gender']

emp_df = spark.createDataFrame(data = emp_data,schema = emp_schema)

In [0]:
emp_df.groupBy('dept').agg(sum('sal')).show()

+---------+--------+
|     dept|sum(sal)|
+---------+--------+
|       IT|  345000|
|marketing|  220000|
|    sales|  240000|
+---------+--------+



In [0]:
window = Window.partitionBy('dept')
emp_df.withColumn('Total Salary', sum(col('sal')).over(window)).show(truncate=False)

+---+--------+------+---------+------+------------+
|id |name    |sal   |dept     |gender|Total Salary|
+---+--------+------+---------+------+------------+
|1  |manish  |50000 |IT       |m     |345000      |
|4  |mukesh  |80000 |IT       |m     |345000      |
|8  |rashi   |100000|IT       |f     |345000      |
|9  |aditya  |65000 |IT       |m     |345000      |
|11 |rakhi   |50000 |IT       |f     |345000      |
|3  |raushan |70000 |marketing|m     |220000      |
|6  |nikita  |45000 |marketing|f     |220000      |
|7  |ragini  |55000 |marketing|f     |220000      |
|10 |rahul   |50000 |marketing|m     |220000      |
|2  |vikash  |60000 |sales    |m     |240000      |
|5  |priti   |90000 |sales    |f     |240000      |
|12 |akhilesh|90000 |sales    |m     |240000      |
+---+--------+------+---------+------+------------+



In [0]:
window = Window.partitionBy('dept').orderBy('sal')
emp_df.withColumn('Row Number', row_number().over(window)).show(truncate=False)

+---+--------+------+---------+------+----------+
|id |name    |sal   |dept     |gender|Row Number|
+---+--------+------+---------+------+----------+
|1  |manish  |50000 |IT       |m     |1         |
|11 |rakhi   |50000 |IT       |f     |2         |
|9  |aditya  |65000 |IT       |m     |3         |
|4  |mukesh  |80000 |IT       |m     |4         |
|8  |rashi   |100000|IT       |f     |5         |
|6  |nikita  |45000 |marketing|f     |1         |
|10 |rahul   |50000 |marketing|m     |2         |
|7  |ragini  |55000 |marketing|f     |3         |
|3  |raushan |70000 |marketing|m     |4         |
|2  |vikash  |60000 |sales    |m     |1         |
|5  |priti   |90000 |sales    |f     |2         |
|12 |akhilesh|90000 |sales    |m     |3         |
+---+--------+------+---------+------+----------+



In [0]:
window = Window.partitionBy('dept').orderBy('sal')
emp_df.withColumn('Rank', rank().over(window)).show(truncate=False)

+---+--------+------+---------+------+----+
|id |name    |sal   |dept     |gender|Rank|
+---+--------+------+---------+------+----+
|1  |manish  |50000 |IT       |m     |1   |
|11 |rakhi   |50000 |IT       |f     |1   |
|9  |aditya  |65000 |IT       |m     |3   |
|4  |mukesh  |80000 |IT       |m     |4   |
|8  |rashi   |100000|IT       |f     |5   |
|6  |nikita  |45000 |marketing|f     |1   |
|10 |rahul   |50000 |marketing|m     |2   |
|7  |ragini  |55000 |marketing|f     |3   |
|3  |raushan |70000 |marketing|m     |4   |
|2  |vikash  |60000 |sales    |m     |1   |
|5  |priti   |90000 |sales    |f     |2   |
|12 |akhilesh|90000 |sales    |m     |2   |
+---+--------+------+---------+------+----+



In [0]:
window = Window.partitionBy('dept').orderBy('sal')
emp_df.withColumn('Desne Rank', dense_rank().over(window)).show(truncate=False)

+---+--------+------+---------+------+----------+
|id |name    |sal   |dept     |gender|Desne Rank|
+---+--------+------+---------+------+----------+
|1  |manish  |50000 |IT       |m     |1         |
|11 |rakhi   |50000 |IT       |f     |1         |
|9  |aditya  |65000 |IT       |m     |2         |
|4  |mukesh  |80000 |IT       |m     |3         |
|8  |rashi   |100000|IT       |f     |4         |
|6  |nikita  |45000 |marketing|f     |1         |
|10 |rahul   |50000 |marketing|m     |2         |
|7  |ragini  |55000 |marketing|f     |3         |
|3  |raushan |70000 |marketing|m     |4         |
|2  |vikash  |60000 |sales    |m     |1         |
|5  |priti   |90000 |sales    |f     |2         |
|12 |akhilesh|90000 |sales    |m     |2         |
+---+--------+------+---------+------+----------+



In [0]:
window = Window.partitionBy('dept').orderBy('sal')


emp_df.withColumn('Row Number', row_number().over(window))\
    .withColumn('Rank', rank().over(window))\
    .withColumn('Desne Rank', dense_rank().over(window)).show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |sal   |dept     |gender|Row Number|Rank|Desne Rank|
+---+--------+------+---------+------+----------+----+----------+
|1  |manish  |50000 |IT       |m     |1         |1   |1         |
|11 |rakhi   |50000 |IT       |f     |2         |1   |1         |
|9  |aditya  |65000 |IT       |m     |3         |3   |2         |
|4  |mukesh  |80000 |IT       |m     |4         |4   |3         |
|8  |rashi   |100000|IT       |f     |5         |5   |4         |
|6  |nikita  |45000 |marketing|f     |1         |1   |1         |
|10 |rahul   |50000 |marketing|m     |2         |2   |2         |
|7  |ragini  |55000 |marketing|f     |3         |3   |3         |
|3  |raushan |70000 |marketing|m     |4         |4   |4         |
|2  |vikash  |60000 |sales    |m     |1         |1   |1         |
|5  |priti   |90000 |sales    |f     |2         |2   |2         |
|12 |akhilesh|90000 |sales    |m     |3         |2   |2         |
+---+-----

# Spark SQL

In [0]:
flight_df.createOrReplaceTempView("Flight_view")

In [0]:
%sql
select * from Flight_view

In [0]:
spark.sql("select * from {flight_df}",flight_df = flight_df).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
spark.sql("""
          select *,
          case when age<18 then 'Minor'
          when age>18 then 'Major'
          else 'Novalue'
          end as Adult
          from {emp_df}
          
          """,emp_df = emp_df).show()

+----+-------+----+------+-------+-----------+-------+
|  id|   name| age|salary|country|       dept|  Adult|
+----+-------+----+------+-------+-----------+-------+
|   1| manish|  26| 20000|  india|         IT|  Major|
|   2|  rahul|null| 40000|germany|engineering|Novalue|
|   3|  pawan|  12| 60000|  india|      sales|  Minor|
|   4|roshini|  44|  null|     uk|engineering|  Major|
|   5|raushan|  35| 70000|  india|      sales|  Major|
|   6|   null|  29|200000|     uk|         IT|  Major|
|   7|   adam|  37| 65000|     us|         IT|  Major|
|   8|  chris|  16| 40000|     us|      sales|  Minor|
|null|   null|null|  null|   null|       null|Novalue|
|   7|   adam|  37| 65000|     us|         IT|  Major|
+----+-------+----+------+-------+-----------+-------+

